# LLaMA-Factory 微调大模型

首先，我们来安装 [LLaMA-Factory](https://github.com/hiyouga/LLaMA-Factory)。LLaMA Factory 是一个比较容易上手的微调框架，可以通过 WebUI 来零代码微调大模型。

## 1. 环境准备

### 1.1 安装 LLaMA Factory

**1）安装 CUDA**

参考：[CUDA 安装](https://llamafactory.readthedocs.io/zh-cn/latest/getting_started/installation.html#cuda)

运行以下命令检查驱动是否安装：

```bash
nvidia-smi
```

PS：假定你已经安装好驱动，驱动的安装方法这里略过。

安装 CUDA Toolkit：

```bash
# 安装 build-essential 和 gcc-multilib
sudo apt-get update
sudo apt-get install build-essential
sudo apt-get install gcc-multilib

# 设置 CUDA 仓库
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
sudo dpkg -i cuda-keyring_1.1-1_all.deb

# 更新并安装 CUDA
sudo apt-get update
sudo apt-get install cuda
```

**2）安装 LLaMA-Factory**

运行以下命令，安装 LLaMA-Factory：

```bash
# 创建一个文件夹放 repository
mkdir envs && cd envs

# 下载 LLaMA-Factory 代码仓库
git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

# 安装 LLaMA-Factory 及其依赖
cd LLaMA-Factory
pip install -e ".[torch,metrics]"
```

检查安装：

```bash
# 查看版本
llamafactory-cli version

# 查看训练相关的参数帮助
llamafactory-cli train -h
```

检查 pytorch 环境：

```python
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.__version__
```

**3）安装 bitsandbytes**

如果想启用量化 LoRA（QLoRA），需要安装 bitsandbytes。

在 **Linux** 上安装 bitsandbytes：

```bash
pip install bitsandbytes
```

如果想在 **Windows** 上启用 QLoRA，请根据 CUDA 版本选择合适的 [bitsandbytes](https://github.com/jllllll/bitsandbytes-windows-webui/releases/tag/wheels/) 发行版。

执行以下命令，查看 CUDA 版本:

```bash
nvcc --version
```

我的 CUDA 版本是 12.6

```bash
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Sep_12_02:55:00_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.77
Build cuda_12.6.r12.6/compiler.34841621_0
```

选择安装较新的版本 0.41.1：

```bash
pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.1-py3-none-win_amd64.whl
```

### 1.2 下载 Qwen 模型

前几个项目用的都是 Qwen 模型，秉持着推理代码可以复用的偷懒精神，这次还用 Qwen。我有 8G 显存，当前最大可接受的参数量是 7B，阿里经过指令微调的 `Qwen2.5-7B-Instruct` 模型是我最好的选择，后续开 QLoRA 可将显存消耗压制在合适范围内。

多大模型用什么训练方式需要多大的 GPU 可参考：[hardware-requirement](https://github.com/hiyouga/LLaMA-Factory?tab=readme-ov-file#hardware-requirement)

如果显存不足，可以选择 1.5B 或 3B 模型：

- [Qwen/Qwen2.5-1.5B-Instruct](https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct)
- [Qwen/Qwen2.5-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct)

**1）安装 ModelScope**

```bash
pip install modelscope
pip install modelscope-agent
```

**2）下载 Qwen2.5-7B-Instruct 模型**

中国大陆建议使用 `ModelScope` 下载模型：

```python
from modelscope import snapshot_download

model_dir = snapshot_download('Qwen/Qwen2.5-7B-Instruct', cache_dir='./')
print(f"model_dir: {model_dir}")
```

> 以上代码同本仓库 [download_qwen.py](/model/download_qwen.py) 脚本。
> 
> 可在根目录打开命令行，执行 `cd model && python download_qwen.py` 下载 Qwen 模型。

### 1.3 模型推理测试

模型下载好之后，测试一下模型能否正常推理。

**1）使用 transformers 库推理**

In [1]:
import os
import torch
import transformers

MODEL_PATH = "./model/Qwen/Qwen2.5-7B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cuda


In [2]:
# 模型的绝对路径
abs_model_path = os.path.abspath(MODEL_PATH)

pipeline = transformers.pipeline(
    "text-generation",
    model=abs_model_path,
    model_kwargs={
        "torch_dtype": torch.bfloat16
    },
    device_map=device,  # auto cuda
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=pipeline.tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


Arrr, I be yer trusty pirate chatbot, ready to sail the cyber seas wit ye and share tales of treasure and treachery! What be yer name, matey?


**2）使用 ChatBot 推理**

LLaMA Factory 提供了一个基于 [gradio](https://github.com/gradio-app/gradio) 开发的 ChatBot 推理页面。

运行以下命令启动 ChatBot：

```bash
model_path="./model/Qwen/Qwen2.5-7B-Instruct"
CUDA_VISIBLE_DEVICES=0 llamafactory-cli webchat \
    --model_name_or_path $model_path\
    --template qwen
```

> PS: 如需了解 template 的枚举值，可查看对话模板文件：[template.py](https://github.com/hiyouga/LLaMA-Factory/blob/main/src/llamafactory/data/template.py)

模型加载完毕后，浏览器打开 [http://localhost:7861/](http://localhost:7861/) 即进入推理页。

![webchat_init](./img/webchat_init.jpg)

## 2. 数据准备

模型微调后用于何种任务，决定了应该使用什么数据集。业界常见的任务包括：NL2SQL，商品描述生成、广告文案生成等。但是以上任务班味太重，不适合我们 side project 的调性。经过一番挑三拣四，决定用医疗对话数据集来优化 Qwen 模型在问诊场景的效果。

### 2.1 下载医疗对话数据集

数据集 **shibing624/medical** 是医疗大模型 [shibing624/MedicalGPT](https://github.com/shibing624/MedicalGPT) 的训练数据集，

- 从 Huggingface 下载：[shibing624/medical](https://huggingface.co/datasets/shibing624/medical/tree/main/finetune)
- 从 Modelscope 下载：[medical_zh](https://modelscope.cn/datasets/swift/medical_zh)

无需下载全部数据文件，只需要下载以下 3 个和中文 SFT 有关的数据文件即可。下载后放在本项目的 `/data` 目录：

- `train_zh_0.json`
- `test_zh_0.json`
- `valid_zh_0.json`

### 2.2 检查数据格式

使用以下命令获取 `train_zh_0.json` 文件的前三行：

```bash
cd data
head -3 train_zh_0.json
```

样例数据如下：

```bash
{"instruction": "血热的临床表现是什么?", "input": "", "output": "初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。"}
{"instruction": "帕金森叠加综合征的辅助治疗有些什么？", "input": "", "output": "综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗"}
{"instruction": "卵巢癌肉瘤的影像学检查有些什么？", "input": "", "output": "超声漏诊；声像图；MR检查；肿物超声；术前超声；CT检查"}
```

非常棒，数据已经是 alpaca 格式，满足 LLaMA Factory 的数据格式要求，无需进行数据格式转换。

### 2.3 添加描述文件

使用 `wc -l train_zh_0.json` 查看文件行数，发现训练集有 194 万多行，太多了。为了快速跑完第一次微调任务，从训练集中抽取前 1000 条样本：

```bash
head -1000 train_zh_0.json > train_zh_1000.json
```

新建数据集描述文件 `data/dataset_info.json`，参考 [格式要求](https://llamafactory.readthedocs.io/zh-cn/latest/getting_started/data_preparation.html#id4) 填写：

```json
{
  "train_zh": {
    "file_name": "train_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "train_zh_1000": {
    "file_name": "train_zh_1000.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "test_zh": {
    "file_name": "test_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "valid_zh": {
    "file_name": "valid_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  }
}
```

## 3. 微调大模型

以上，我们已经准备了微调框架、模型文件、微调数据集，下面开始微调模型。

打开命令行，来到项目根目录，执行以下命令启动 WebUI 页面：

```bash
llamafactory-cli webui

# 如需指定显卡 id 和 WebUI 端口
# CUDA_VISIBLE_DEVICES=0 GRADIO_SERVER_PORT=7860 llamafactory-cli webui
```

### 3.1 SFT 监督微调

配置好模型路径、微调方法、数据集等参数。由于我的显存限制，开启了 4-bit 量化，会损失一些训练精度。

![webui_train](./img/webui_train.png)

我的训练参数如下，供参考：

|参数名|参数值|
| -- | -- |
|模型名称|Qwen2.5-7B-Instruct|
|模型路径|model/Qwen/Qwen2___5-7B-Instruct|
|微调方法|lora|
|量化等级|4|
|量化方法|bitsandbytes|
|训练阶段|Supervised Fine-Tuning|
|数据路径|data|
|数据集|train_zh_1000|

使用 `预览命令` 功能，生成当前训练参数下的训练脚本：

```bash
llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path model/Qwen/Qwen2___5-7B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset train_zh_1000 \
    --cutoff_len 2048 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 100000 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2.5-7B-Instruct/lora/train_2025-04-11-15-29-12 \
    --bf16 True \
    --plot_loss True \
    --trust_remote_code True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --optim adamw_torch \
    --quantization_bit 4 \
    --quantization_method bitsandbytes \
    --double_quantization True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all 
```

训练参数配置好后，点击“开始”进行训练。由于我们只选择了 1000 条样本进行训练，训练很快就能结束。

训练中，loss 逐渐下降：

![webui_training](./img/webui_training.jpg)

8G 显存接近跑满：

![gpu_stat](./img/gpu_stat.png)

> PS: 为了防止训练因 Windows 的电源计划或睡眠设置中断，可以启用 [PowerToys](https://github.com/microsoft/PowerToys/releases/latest) 的 [唤醒功能](https://learn.microsoft.com/en-us/windows/powertoys/awake)。它支持一键维持电脑的唤醒状态。

### 3.2 加载训练好的 LoRA 文件

训练完毕后, 点击“检查点路径”，即可在下拉栏中找到该模型历史上使用 WebUI 训练的 LoRA 模型。点选我们上一次训练的检查点，后续再训练或者执行 chat 的时候，会将此 LoRA 一起加载。

![webui_chat](./img/webui_chat.png)

加载好之后，我们来问一个问题，看一下它回答得怎么样：`3 个月先兆流产怎么办`

![webui_question](./img/webui_question.png)

作为对比，下面是不加载 LoRA 的原模型的回答：

![webui_question2](./img/webui_question2.png)

这个问题是根据训练数据集中的这个样本提出的：

```
{"instruction": "五个月先兆流产怎么办", "input": "", "output": "五个月先兆流产应该是有自然流产引起的，引起的原因可能是有胎儿停育或者有孕期感染及用药或者外伤剧烈运动等原因引起的建议，你现在先注意观察出血，最好是及时去医院做检查查看，然后再进行清宫手术的，并注意术后的休息及卫生等。怀孕五个月在家突然引起流产的情况，这个是比较危险的，导致大出血的现象，建议您需要及时的去医院妇产科就诊。流产后一定要注意休息，饮食吃一些营养丰富的食物，一个月内禁止同房，注意外阴部位的清洁卫生，避免细菌侵入，引起感染。怀孕五个月，有先兆流产，需要卧床静养，保胎药物，感觉胎动位置偏下，可以超声检查胎儿在宫内的活动情况，也要测量宫颈管长度，避免宫颈机能不全。胎儿的胎动没有规律性，有时频繁一点，也不是异常情况，大多在傍晚或者夜晚的时候感觉比较明显一些，使用胎心仪听诊胎心搏动，比较规律就没有太大的问题。若是预防入盆的可能不大，建议彩超的情况观察是否胎盘低导致的，注意少食生冷刺激性食物，建议定期到医院检查胎儿的情况保胎用药的。目前的情况是绝对卧床休息，避免劳累。放松心情，不要过于担心。注意定期产前检查。左侧卧位，有利于胎盘供氧。注意胎心及胎动情况。您需要去当地医院做B超看一下，然后在医生的指导下积极进行保胎治疗。建议您暂时尽量多卧床休息，避免过度劳累，避免剧烈运动，在家人的陪同下稳稳的去医院检查，检查过后在医生的指导下按时服药，进行保胎治疗。如果是胎儿本身问题引起的流产，需要及时终止妊娠，术后要多吃有营养的食物，禁食辛辣刺激性食物，多注意休息，避免剧烈运动。流产不用担心，半年后可以再次备孕。"}
```

你觉得大模型的回答有几分神韵？

### 3.3 导出微调后的模型

WebUI 支持将训练后的 LoRA 和原始大模型进行融合，输出一个完整的模型文件。

导出操作也很简单。首先切换到 `Export` 分页，然后配置好导出路径和检查点路径，点击“开始导出”，少时就导出完成了。

![webui_export](./img/webui_export.png)

导出后的模型目录包含以下文件：

```
.
├── Modelfile
├── added_tokens.json
├── config.json
├── generation_config.json
├── merges.txt
├── model-00001-of-00004.safetensors
├── model-00002-of-00004.safetensors
├── model-00003-of-00004.safetensors
├── model-00004-of-00004.safetensors
├── model.safetensors.index.json
├── special_tokens_map.json
├── tokenizer.json
├── tokenizer_config.json
└── vocab.json
```

## 4. vLLM 作为推理后端

### 4.1 启动后端推理服务

使用 LLaMA Factory 作为推理后端，需要安装 vLLM 环境。安装过程可参考我的上一篇博客 [《本地部署大模型：Ollama 和 vLLM》](https://www.luochang.ink/posts/llm_deploy/#%E4%B8%80%E6%9C%AC%E5%9C%B0%E9%83%A8%E7%BD%B2-deepseek-r1)

```bash
# 打开安装好 vLLM 的虚拟环境
conda activate vllm_env

# 补充安装 LLaMA Factory
cd ./envs/LLaMA-Factory/
pip install -e ".[torch,metrics]"
```

安装好环境后，运行下面这个脚本启动推理服务：

```bash
#!/bin/bash
# USAGE: bash server.sh

source $(conda info --base)/etc/profile.d/conda.sh
conda activate vllm_env

# 我的电脑只支持运行 0.5B 模型
# 如果你也是，请先用 ./model/download_qwen.py 下载模型
model_path="./model/Qwen/Qwen2___5-0___5B-Instruct"
CUDA_VISIBLE_DEVICES=0 API_PORT=8621 llamafactory-cli api \
    --model_name_or_path $model_path \
    --template qwen \
    --infer_backend vllm \
    --vllm_gpu_util 0.99 \
    --vllm_maxlen 512 \
    --vllm_enforce_eager
```

### 4.2 运行客户端获取结果

上一篇博客写好的 Qwen 客户端代码 [qwen_vllm_bash_client.py](https://github.com/luochang212/llm-deploy/blob/main/server/qwen_vllm_bash_client.py)，这里直接拿来用：

In [3]:
# -*- coding: utf-8 -*-

from openai import OpenAI


openai_api_base = "http://localhost:8621/v1"


def chat_completion(prompt, model=''):  # llama factory 不需要写模型名，空着就行
    client = OpenAI(
        base_url=openai_api_base,
        api_key='EMPTY_KEY'  # 也不需要写 api key，非空/非空字符串就行
    )

    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.8,
        top_p=0.9,
        max_tokens=512,
        extra_body={
            "repetition_penalty": 1.05,
        },
    )

    return chat_response

In [4]:
response = chat_completion(prompt="抑郁症有哪些症状")
content = response.choices[0].message.content
print(content)

抑郁症是一种常见的心理健康问题，其症状可能包括但不限于以下几点：

1. **情绪低落**：持续的悲伤、空虚或快乐感丧失。
2. **兴趣丧失**：对以前喜欢的活动失去兴趣或乐趣。
3. **能量下降**：精力减退，无法应对日常活动。
4. **疲劳**：持续的疲倦或缺乏精力。
5. **睡眠障碍**：难以入睡或早醒，或者睡眠模式改变。
6. **食欲变化**：食欲增加或减少，导致体重变化。
7. **注意力和记忆力问题**：难以集中注意力，记忆力减退。
8. **自杀念头或行为**：有自杀的想法或行为，可能包括企图自杀。
9. **身体症状**：头痛、肌肉疼痛、睡眠问题、胃痛、疲劳等。

如果怀疑自己或身边的人有抑郁症的症状，建议寻求专业心理医生的帮助，通过专业的评估和治疗来获得适当的帮助和支持。抑郁症是可以治疗的，通过适当的治疗和生活方式的调整，许多抑郁症患者可以恢复健康。
